In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel

In [4]:
from collections import Counter

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [5]:
path = Path('data/midi')

In [6]:
source_dir = 'midi_transform_v2'
out_dir = 'midi_numpy_v2'
source_csv = path/source_dir/f'{source_dir}.csv'
out_csv = path/out_dir/f'{out_dir}.csv'

In [7]:
df = pd.read_csv(source_csv, index_col=0); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_offset,ht_time_signature,inferred_key,inferred_offset,...,midi,midi_title,parts,seconds,section,song_url,source,time_signature,title,midi_transform_v2
index,,,,,,,,,,,,,,,,,,,,,
0,wayne-sharpe,128.0,NaN,128.0,C,1.0,0.0,4.0,C major,0.0,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,yu-gi-oh3,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song,data/midi/midi_transform_v2/hooktheory/pianoro...
1,weezer,NaN,NaN,128.0,C,NaN,0.0,4.0,NaN,NaN,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,My New Song,intro-and-verse,NaN,intro-and-verse,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,NaN,beverly-hills,NaN
2,weezer,108.0,NaN,108.0,Eb,1.0,-3.0,4.0,E- major,-3.0,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,falling for you intro,"intro,verse,chorus,solo",11.111111,intro,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,falling-for-you,data/midi/midi_transform_v2/hooktheory/pianoro...
3,weezer,121.0,"Pop,Rock",121.0,Ab,1.0,4.0,4.0,A- major,4.0,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,Buddy Holly,"verse,pre-chorus,chorus,bridge,solo",43.388430,solo,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,buddy-holly,data/midi/midi_transform_v2/hooktheory/pianoro...
4,wavves,180.0,NaN,180.0,A,1.0,3.0,4.0,B minor,-2.0,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,dreams of grandeur,verse,21.333333,verse,https://www.hooktheory.com/theorytab/view/wavv...,hooktheory,4/4,dreams-of-grandeur,data/midi/midi_transform_v2/hooktheory/pianoro...


In [8]:
df_filtered = df.loc[df['time_signature'] == '4/4']; df_filtered.shape

(30201, 21)

In [9]:
def transcribe_file(idxrow):
    idx,row = idxrow
    midi_file = row[source_dir]
    if not isinstance(midi_file, str) or not Path(midi_file).exists(): return idx,None
    out_file = Path(midi_file.replace(f'/{source_dir}/', f'/{out_dir}/')).with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    try:
        stream = file2stream(midi_file) # 1.
        chordarr = stream2chordarr(stream) # 2.
        if row.source != 'hooktheory': # keep hooktheory
            chord_short = remove_chordarr_rests(trim_chordarr_rests(chordarr))
            delta_trim = chordarr.shape[0] - chord_short.shape[0]
            if delta_trim > 100: print(f'Removed {delta_trim} rests from {midi_file}')
            chordarr = chord_short
        pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)
        np.save(out_file, chordarr)
    except Exception as e:
        print('Error converting midi to sequence', e)
        return idx,None
    return idx,str(out_file)

In [10]:
# for r in df_filtered.iterrows():
#     transcribe_file(r)

In [11]:
transcribed_files = process_parallel(transcribe_file, df_filtered.iterrows(), total=df_filtered.shape[0])

Error converting midi to sequence 
Error converting midi to sequence 
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127
Error converting midi to sequence index 903 is out of bounds for axis 0 with size 903
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127
Error converting midi to sequence index 4707 is out of bounds for axis 0 with size 4707


KeyboardInterrupt: 

In [ ]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [ ]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

In [ ]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

### Calculate timesteps

In [ ]:
merged_df = pd.read_csv(out_csv)

In [ ]:
def calc_timesteps(idxrow):
    idx,row = idxrow
    np_file = row[out_dir]
    if not isinstance(np_file, str) or not Path(np_file).exists(): return idx,None
    try:
        np.load(np_file)
        timesteps = np.load(np_file).shape[0]
        return idx,timesteps
    except Exception as e:
        print('Error reading text', e)
    return idx, None

In [ ]:
file2steps = process_parallel(calc_timesteps, merged_df.iterrows(), total=merged_df.shape[0])

In [ ]:
len_df = pd.DataFrame(data={f'{out_dir}_timesteps': list(file2steps.values())}, index=list(file2steps.keys()))
merged_len_df = merged_df.join(len_df, how='outer');
len_df.shape, merged_df.shape, merged_len_df.shape

In [ ]:
merged_len_df.to_csv(out_csv, index=False); merged_len_df.head()